# Segment 1 - Machine Learning Model Mock-up 
### Sam Boester 10/23/2022

In [2]:
pip install psycopg2

Note: you may need to restart the kernel to use updated packages.


In [3]:
# dependencies

import pandas as pd
import psycopg2 as pg
import plotly.express as px
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.model_selection import train_test_split
from collections import Counter
from imblearn.ensemble import BalancedRandomForestClassifier

In [ ]:

# Posgres Connection

# engine = pg.connect("dbname='rental_db' user='postgres' host='127.0.0.1' port='5432' password='XXXXXX'")
# df = pd.read_sql('select * from actor', con=engine)

# Confirming that dataframe created.
# df.head()

In [5]:
# Loading Mock CVS, this will be updated to a posgres connection in future segments

# read in CVS


file_path = "resources\Vietnam_USAF_Losses_Cleaned_Data.csv"

usaf_df = pd.read_csv(file_path)

usaf_df.head()




,Crash Date,Crash Time,Aircraft Type,Aircraft S/N,Base,Wing,Squadron,Call Sign,Operations Code,Mission Type,...,Pilot Egress,Pilot Status,Pilot Condition,Pilot Recovered,Co-Pilot/Nav Rank,Co-Pilot/Nav,Co-Pilot/Nav Egress,Co-Pilot/Nav Status,Co-Pilot/Nav Condition,Co-Pilot Recovered
0,620202,_,C-123B,564370,TSN,464TCW,_,_,IC,NaN,...,Crash,KIA,_,_,Capt,Larsen R. D.,Crash,KIA,_,_
1,620211,_,SC-47A,4315732,BHA,_,4400CCTS,_,IC,NaN,...,Crash,KIA,_,_,Capt,Hartson S. G.,Crash,KIA,_,_
2,620613,_,T-28B,0,_,_,_,_,IC,NaN,...,_,Recovered,Uninjured,_,u,_,_,_,_,_
3,620828,_,T-28B,538376,_,_,_,_,IC,NaN,...,_,KIA,_,_,Capt,_,_,_,_,_
4,621015,_,U-10,625909,_,_,_,_,IC,NaN,...,Crash,KIA,_,_,Capt,Foxx R. L.,Crash,KIA,_,_


## Pre-Model Cleaning

In [ ]:
# Capture capture variables for model". 

usaf_df_model = usaf_df.filter(['Crash Date','Aircraft Type','Base', 'Hit Country', 'Loss Longitude', 'Loss Latitude', 'Pilot Status'], axis=1)

usaf_df_model.head()

In [ ]:
# remove spaces from column names"
usaf_df_model.columns = usaf_df_model.columns.str.replace(' ', '')

usaf_df_model.head()

In [ ]:
# Remove rows that have string value "_".  For some reason I couldnt chanin the code to do all columns in one line

#update to a loop? like the below sample code?

#for col in cols_to_check:
    #df[col] = df[col].map(lambda x: x.replace(';',''))

usaf_df_modelv2 = usaf_df_model[usaf_df_model["LossLongitude"].str.contains("_")==False]

usaf_df_modelv3 = usaf_df_modelv2[usaf_df_modelv2["AircraftType"].str.contains("_")==False]

usaf_df_modelv4 = usaf_df_modelv3[usaf_df_modelv3["Base"].str.contains("_")==False]

usaf_df_modelv5 = usaf_df_modelv4[usaf_df_modelv4["HitCountry"].str.contains("_")==False]

usaf_df_modelv6 = usaf_df_modelv5[usaf_df_modelv5["PilotStatus"].str.contains("_")==False]

usaf_cleaned_nulls = usaf_df_modelv6[usaf_df_modelv6["LossLatitude"].str.contains("_")==False]

usaf_cleaned_nulls.head(20)

In [ ]:
#Start with dummy coding text variables

usaf_df_dummy = pd.get_dummies(usaf_cleaned_nulls, columns=["AircraftType", "Base", "HitCountry"])

usaf_df_dummy.head()

In [ ]:
# Clean Target variable pilot status - Get labels and counts

usaf_df_dummy["PilotStatus"].value_counts()



In [ ]:
# Clean Target variable pilot status - Create Binomial with dictionary.map

recovered = {'Recovered': 1, 'KIA': 0,'POW (returned)': 1,'MIA': 0,'POW (died)': 0,'POW': 1,'KIA (chute failure)': 0,'Recoverd': 1, 'u': 0, 
    'KIA (chute failed)': 0, 'Recovered DaNang': 1, 'POW (died in captivity)':0, 'Recovered (chute fail)': 1,'recovered': 1,'KIA,body MIA,PJ abandnd': 1, 'POW (died in captivity)': 0}


usaf_df_dummy["Target"] = usaf_df_dummy["PilotStatus"].map(recovered)

usaf_df_dummy["Target"].value_counts

## Random Forest Draft

### During the assignment we will test different supervised models to determine,
### if pilot survival could be predicted by aircraft, artillery, and location data from the Vietnam War. 

In [ ]:
# Create feature Variables

Y = usaf_df_dummy["Target"]

x = usaf_df_dummy.drop(["Target", "PilotStatus"], axis=1)

X.head()

In [ ]:
# Check the balance of our target values - We have greater than 1000 and the target of recovered versus non-recovered is roughly proportional
y.value_counts()

In [ ]:
# Splitting into Train and Test sets - modify
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [ ]:
# Resample the training data with the BalancedRandomForestClassifier - may need to change estimator count
# create model instance

brfc = BalancedRandomForestClassifier(n_estimators=100, random_state=1)

modle=brfc.fit(X_train, y_train)

In [ ]:
# Calculated the balanced accuracy score
y_pred = brfc.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

In [ ]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

In [ ]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

# Logistic Reression Draft

#### For a basic logistic regression model we will test different sampling techniques through out the assignment

Random Oversampling

In [65]:
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

Counter(y_resampled)

Counter({1: 566, 0: 566})

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='lbfgs', random_state=1)

model.fit(X_resampled, y_resampled)

In [ ]:
from sklearn.metrics import balanced_accuracy_score

y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

In [ ]:
confusion_matrix(y_test, y_pred)

In [ ]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, y_pred))

SMOTE Oversampling

In [ ]:
from imblearn.over_sampling import SMOTE

X_resampled, y_resampled = SMOTE(random_state=1, sampling_strategy=1.0).fit_resample(
    X_train, y_train
)
from collections import Counter

Counter(y_resampled)

In [ ]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

In [ ]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

In [ ]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

In [ ]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

In [ ]:
# Resample the data using the ClusterCentroids resampler

from imblearn.under_sampling import ClusterCentroids

cluster = ClusterCentroids(random_state=1)
X_resampled, y_resampled = cluster.fit_resample(X_train, y_train)

Counter(y_resampled)

In [ ]:
# Train Model

model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

In [ ]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

In [ ]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

In [ ]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

### SMOTEEN

In [ ]:
from imblearn.combine import SMOTEENN

smoteenn = SMOTEENN(random_state=1)
X_resampled, y_resampled = smoteenn.fit_resample(X_train, y_train)
Counter(y_resampled)

In [ ]:
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

In [ ]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

In [ ]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

In [ ]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))